# Dev

In [2]:
import json
import os

files = ['dev', 'train']

# Đường dẫn đến file JSON lớn của bạn
for file in files: 
    input_file = rf'D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\{file}.json'
    # Thư mục DATA bạn đã thiết lập
    output_dir = rf'D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\{file}'

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(input_file, 'r', encoding='utf-8') as f:
        big_data = json.load(f)

    # Lặp qua từng bài viết và lưu thành file riêng lẻ
    for i, (key, value) in enumerate(big_data.items()):
        # Chuyển đổi cấu trúc source từ dict {"0": [...]} thành list [...]
        article_sents = value['source']['0']
        abstract_text = value['target']
        
        # Tạo object mới đúng cấu trúc script mong đợi
        single_data = {
            "source": article_sents,  # Script của bạn đang dùng key 'source'
            "target": [abstract_text] # Đưa target vào list vì script tokenize theo list
        }
        
        with open(os.path.join(output_dir, f'{i}.json'), 'w', encoding='utf-8') as f:
            json.dump(single_data, f, indent=4, ensure_ascii=False)

    print(f"Đã tạo xong {len(big_data)} file JSON tại {output_dir}")

Đã tạo xong 1892 file JSON tại D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\dev
Đã tạo xong 2000 file JSON tại D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\train


In [3]:
import json
import os

files = ['test']

# Đường dẫn đến file JSON lớn của bạn
for file in files: 
    input_file = rf'D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\{file}.json'
    # Thư mục DATA bạn đã thiết lập
    output_dir = rf'D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\{file}'

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(input_file, 'r', encoding='utf-8') as f:
        big_data = json.load(f)

    # Lặp qua từng bài viết và lưu thành file riêng lẻ
    for i, (key, value) in enumerate(big_data.items()):
        # Chuyển đổi cấu trúc source từ dict {"0": [...]} thành list [...]
        article_sents = value['source']['0']
        abstract_text = value['target']
        
        # Tạo object mới đúng cấu trúc script mong đợi
        single_data = {
            "source": article_sents,  # Script của bạn đang dùng key 'source'
            "target": [abstract_text] # Đưa target vào list vì script tokenize theo list
        }
        
        with open(os.path.join(output_dir, f'{i}.json'), 'w', encoding='utf-8') as f:
            json.dump(single_data, f, indent=4, ensure_ascii=False)

    print(f"Đã tạo xong {len(big_data)} file JSON tại {output_dir}")

Đã tạo xong 3789 file JSON tại D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\test


# vocab_cnt

In [4]:
import os
import json
import pickle
from collections import Counter
from os.path import join

# Trỏ đến thư mục dataset của bạn
DATA_DIR = r'D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset'

def build_vocab():
    word_cnt = Counter()
    # Chỉ nên quét tập train để xây dựng từ điển
    split_dir = join(DATA_DIR, 'train')
    
    if not os.path.exists(split_dir):
        print(f"Lỗi: Không tìm thấy thư mục {split_dir}")
        return

    print("Đang quét dữ liệu để tạo vocab_cnt.pkl...")
    files = [f for f in os.listdir(split_dir) if f.endswith('.json')]
    
    for filename in files:
        with open(join(split_dir, filename), 'r', encoding='utf-8') as f:
            data = json.load(f)
            # Theo cấu trúc Wikilingual bạn đã gửi:
            # data['source'] là list các câu, data['target'] là string hoặc list
            articles = data.get('source', [])
            abstracts = data.get('target', [])
            if isinstance(abstracts, str): abstracts = [abstracts]
            
            for sent in articles + abstracts:
                word_cnt.update(sent.lower().split())

    # Lưu file pkl
    with open(join(DATA_DIR, 'vocab_cnt.pkl'), 'wb') as f:
        pickle.dump(word_cnt, f)
    
    print(f"Hoàn thành! Đã tạo file tại: {join(DATA_DIR, 'vocab_cnt.pkl')}")
    print(f"Tổng số từ vựng: {len(word_cnt)}")

if __name__ == '__main__':
    build_vocab()

Đang quét dữ liệu để tạo vocab_cnt.pkl...
Hoàn thành! Đã tạo file tại: D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\vocab_cnt.pkl
Tổng số từ vựng: 13335


In [8]:
import os
import json
import pickle
import tarfile
import re
from collections import Counter
from os.path import join
import unicodedata

# Đường dẫn thư mục dữ liệu trên Linux của bạn
DATA_DIR = "D:\\NguyenTienDat_23520262\\Nam_3\\ViPhoLM_DoAn\\datasets\\Wikilingual-dataset"

def simple_tokenize(sentence):
    sentence = sentence.lower()
    sentence = unicodedata.normalize("NFC", sentence)
    # remove all non-characters and punctuations
    sentence = re.sub(r"[-/“”!\*\&\$\.\?:;,\"'\(\[\]\(\)]", " ", sentence)
    sentence = " ".join(sentence.strip().split()) # remove duplicated spaces
    tokens = sentence.strip().split()
    
    return tokens

def process_and_build():
    word_cnt = Counter()
    splits = ['train', 'dev', 'test']
    
    for split in splits:
        input_file = join(DATA_DIR, f"{split}.json")
        output_dir = join(DATA_DIR, split)
        if not os.path.exists(output_dir): 
            os.makedirs(output_dir)
        
        print(f"Đang xử lý tập {split} (Tách từ đơn)...")
        
        # Kiểm tra file input
        if not os.path.exists(input_file):
            print(f"Bỏ qua {split} vì không tìm thấy file .json")
            continue

        with open(input_file, 'r', encoding='utf-8') as f:
            big_data = json.load(f)
            
        for i, (key, value) in enumerate(big_data.items()):
            # Lấy dữ liệu nguồn và mục tiêu
            # Giả sử cấu trúc: source: {"0": ["câu 1", "câu 2"]}, target: "chuỗi tóm tắt"
            raw_source = value['source'].get('0', []) if isinstance(value['source'], dict) else value['source']
            raw_target = value['target']

            # Tách thành từng từ đơn
            source_sents = [" ".join(simple_tokenize(s)) for s in raw_source]
            # Target phải để trong List theo yêu cầu của repo fast_abs_rl
            target_text = [" ".join(simple_tokenize(raw_target))]
            
            # Cập nhật từ điển Vocab (tách nhỏ ra để đếm từng từ)
            for sent in source_sents + target_text:
                word_cnt.update(sent.split())
            
            # Lưu file JSON nhỏ lẻ
            single_data = {
                "article": source_sents, 
                "abstract": target_text
            }
            with open(join(output_dir, f'{i}.json'), 'w', encoding='utf-8') as f_out:
                json.dump(single_data, f_out, ensure_ascii=False, indent=4)

    # Lưu file vocab_cnt.pkl (đây là file Counter đối tượng)
    with open(join(DATA_DIR, 'vocab_cnt.pkl'), 'wb') as f_vocab:
        pickle.dump(word_cnt, f_vocab)
    print(f"--- Đã tạo xong vocab_cnt.pkl. Tổng từ: {len(word_cnt)} ---")

    # Đóng gói thành Tarballs (.tar)
    for split in splits:
        tar_path = join(DATA_DIR, f"{split}.tar")
        print(f"Đang đóng gói {tar_path}...")
        with tarfile.open(tar_path, "w") as tar:
            # arcname=split giúp cấu trúc bên trong file tar gọn gàng hơn
            tar.add(join(DATA_DIR, split), arcname=split)
            
    print("--- HOÀN TẤT QUY TRÌNH TIỀN XỬ LÝ ---")

if __name__ == '__main__':
    process_and_build()

Đang xử lý tập train (Tách từ đơn)...
Đang xử lý tập dev (Tách từ đơn)...
Đang xử lý tập test (Tách từ đơn)...
--- Đã tạo xong vocab_cnt.pkl. Tổng từ: 17994 ---
Đang đóng gói D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\train.tar...
Đang đóng gói D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\dev.tar...
Đang đóng gói D:\NguyenTienDat_23520262\Nam_3\ViPhoLM_DoAn\datasets\Wikilingual-dataset\test.tar...
--- HOÀN TẤT QUY TRÌNH TIỀN XỬ LÝ ---


In [ ]:
"""produce the dataset with (psudo) extraction label"""
import os
from os.path import exists, join
import json
from time import time
from datetime import timedelta
import multiprocessing as mp

from cytoolz import curry, compose

from utils import count_data
from metric import compute_rouge_l

try:
    OG_DATA_DIR = "D:\\GitHub_Repo\\ViPhoLM\\models\\fast_abstractive\\fast_abs_rl\\data"
except KeyError:
    print('please use environment variable to specify data directories')

try:
    OF_DATA_DIR = "D:\\GitHub_Repo\\ViPhoLM\\models\\fast_abstractive\\fast_abs_rl\\data"
except KeyError:
    print('please use environment variable to specify data directories')

def _split_words(texts):
    return map(lambda t: t.split(), texts)


def get_extract_label(art_sents, abs_sents):
    """ greedily match summary sentences to article sentences"""
    extracted = []
    scores = []
    indices = list(range(len(art_sents)))
    for abst in abs_sents:
        rouges = list(map(compute_rouge_l(reference=abst, mode='r'),
                          art_sents))
        ext = max(indices, key=lambda i: rouges[i])
        indices.remove(ext)
        extracted.append(ext)
        scores.append(rouges[ext])
        if not indices:
            break
    return extracted, scores

@curry
def process(split, i):
    data_dir = join(OG_DATA_DIR, split)
    with open(join(data_dir, '{}.json'.format(i))) as f:
        data = json.loads(f.read())
    tokenize = compose(list, _split_words)
    art_sents = tokenize(data['source'])
    abs_sents = tokenize(data['target'])
    if art_sents and abs_sents: # some data contains empty article/abstract
        extracted, scores = get_extract_label(art_sents, abs_sents)
    else:
        extracted, scores = [], []
    data['extracted'] = extracted
    data['score'] = scores
    with open(join(data_dir, '{}.json'.format(i)), 'w') as f:
        json.dump(data, f, indent=4)

def label_mp(split):
    """ process the data split with multi-processing"""
    start = time()
    print('start processing {} split...'.format(split))
    data_dir = join(OG_DATA_DIR, split)
    n_data = count_data(data_dir)
    with mp.Pool() as pool:
        list(pool.imap_unordered(process(split),
                                 list(range(n_data)), chunksize=1024))
    print('finished in {}'.format(timedelta(seconds=time()-start)))

def label(split):
    start = time()
    print('start processing {} split...'.format(split))
    data_dir = join(OF_DATA_DIR, split)
    n_data = count_data(data_dir)
    for i in range(n_data):
        print('processing {}/{} ({:.2f}%%)\r'.format(i, n_data, 100*i/n_data),
              end='')
        with open(join(data_dir, '{}.json'.format(i))) as f:
            data = json.loads(f.read())
        tokenize = compose(list, _split_words)
        art_sents = tokenize(data['article'])
        abs_sents = tokenize(data['abstract'])
        extracted, scores = get_extract_label(art_sents, abs_sents)
        data['extracted'] = extracted
        data['score'] = scores
        with open(join(data_dir, '{}.json'.format(i)), 'w') as f:
            json.dump(data, f, indent=4)
    print('finished in {}'.format(timedelta(seconds=time()-start)))


def main():
    for split in ['dev', 'train']:  # no need of extraction label when testing
        label_mp(split)

if __name__ == '__main__':
    main()

start processing dev split...
